# 1. Introducción y Objetivos

En el Notebook 1 se realizaron las tareas de preprocesamiento y exploración, donde se cargaron y limpiaron los datos, se seleccionaron los atributos relevantes y se aplicaron transformaciones (discretización) a las señales de conducción de cada combinación de conductor y maniobra. Esto permitió dejar los datos en un estado limpio y organizado, enriquecidos con variables que capturan la tendencia de cada señal a lo largo del tiempo.

El objetivo principal de este Notebook 2 es aplicar la segmentación temporal a dichos datos. Específicamente, se busca:

- **Segmentar** las señales en ventanas deslizantes temporales (con overlapping o sin overlapping) para capturar la dinámica completa de cada maniobra.

- **Etiquetar** cada ventana utilizando la columna *Maneuver marker flag*, determinando si la maniobra se está ejecutando durante ese intervalo.

- **Definir y entrenar un modelo por cada maniobra** de clasificación/detección que, a partir de las características extraídas de cada ventana, identifique la maniobra realizada o detecte su ausencia.

Con este enfoque se pretende aprovechar la alta frecuencia de los datos ($20 \, \text{Hz}$) para analizar la evolución temporal de las maniobras y desarrollar un sistema robusto de detección que será evaluado y ajustado en fases posteriores.

## 1.1. Importación de librerías

In [ ]:
# Asegurar haber instalado las librerías necesarias con requirements.txt (README.md - Apartado 7)

# Importación de las librerías básicas
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configuración básica para los gráficos en línea
%matplotlib inline

# Configurar el estilo de los gráficos
sns.set_theme(style="whitegrid")

## 1.2. Cargado de datos preprocesados

In [2]:
# Directorio base donde se encuentran las carpetas de cada conductor
data_path = "./data"  # Ajusta esta ruta según tu ubicación real

# Lista de nombres de ficheros que esperas encontrar en cada carpeta de Driver
filenames = [
    "STISIMData_3step-Turnings.xlsx",
    "STISIMData_Overtaking.xlsx",
    "STISIMData_Stopping.xlsx",
    "STISIMData_Turnings.xlsx",
    "STISIMData_U-Turnings.xlsx",
]

# Diccionario para almacenar los DataFrames de cada combinación (Driver, Maneuver)
df_dict = {}

# Recorremos cada carpeta en 'data' (Driver1, Driver2, etc.)
for driver_folder in os.listdir(data_path):
    driver_path = os.path.join(data_path, driver_folder)

    # Verificamos que sea una carpeta (para descartar archivos sueltos)
    if os.path.isdir(driver_path):
        # Recorremos la lista de ficheros esperados
        for file in filenames:
            file_path = os.path.join(driver_path, file)

            # Comprobamos que el fichero exista antes de intentar leerlo
            if os.path.exists(file_path):
                # Leemos el archivo Excel
                try:
                    df = pd.read_excel(file_path)

                    # Extraemos el nombre de la maniobra a partir del nombre del fichero, sin la extensión
                    maneuver_name = os.path.splitext(file)[0]

                    # Guardamos el DataFrame en un diccionario con la clave (Driver, Maneuver)
                    df_dict[(driver_folder, maneuver_name)] = df

                except Exception as e:
                    print(f"Error al leer {file_path}: {e}")
            else:
                print(f"Archivo no encontrado: {file_path}")

In [3]:
# Lista de columnas relevantes según el enunciado
relevant_columns = [
    "speed",
    "RPM",
    "Steering wheel angle",
    "Gas pedal",
    "Brake pedal",
    "Clutch pedal",
    "Gear",
    "Maneuver marker flag",
]

# Crear un nuevo diccionario que contiene los DataFrames filtrados por cada (Driver, Maneuver)
df_dict_filtered = {key: df[relevant_columns].copy() for key, df in df_dict.items()}

In [4]:
# Iteramos sobre cada DataFrame en el diccionario filtrado
for key, df in df_dict_filtered.items():

    # 2. Verificar y convertir "Maneuver marker flag" a entero
    if "Maneuver marker flag" in df.columns:
        df["Maneuver marker flag"] = df["Maneuver marker flag"].astype(int)

    # 3. Definir las columnas numéricas relevantes
    numeric_columns = [
        "speed",
        "RPM",
        "Steering wheel angle",
        "Gas pedal",
        "Brake pedal",
        "Clutch pedal",
        "Gear",
    ]

    # 4. Rellenar valores nulos en las columnas numéricas con la mediana
    for col in numeric_columns:
        if df[col].isnull().sum() > 0:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)

    # 5. Convertir las columnas numéricas al tipo float
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")

    print(f"Limpieza para {key} realizada")

Limpieza para ('Driver1', 'STISIMData_3step-Turnings') realizada
Limpieza para ('Driver1', 'STISIMData_Overtaking') realizada
Limpieza para ('Driver1', 'STISIMData_Stopping') realizada
Limpieza para ('Driver1', 'STISIMData_Turnings') realizada
Limpieza para ('Driver1', 'STISIMData_U-Turnings') realizada
Limpieza para ('Driver2', 'STISIMData_3step-Turnings') realizada
Limpieza para ('Driver2', 'STISIMData_Overtaking') realizada
Limpieza para ('Driver2', 'STISIMData_Stopping') realizada
Limpieza para ('Driver2', 'STISIMData_Turnings') realizada
Limpieza para ('Driver2', 'STISIMData_U-Turnings') realizada
Limpieza para ('Driver3', 'STISIMData_3step-Turnings') realizada
Limpieza para ('Driver3', 'STISIMData_Overtaking') realizada
Limpieza para ('Driver3', 'STISIMData_Stopping') realizada
Limpieza para ('Driver3', 'STISIMData_Turnings') realizada
Limpieza para ('Driver3', 'STISIMData_U-Turnings') realizada
Limpieza para ('Driver4', 'STISIMData_3step-Turnings') realizada
Limpieza para ('Driv

In [5]:
# Creamos dos diccionarios:
# - df_dict_original: contendrá una copia de los datos originales.
# - df_dict_discretized: contendrá los datos discretizados, eliminando las columnas originales de las señales.
df_dict_original = {}
df_dict_discretized = {}

for key, df in df_dict_filtered.items():
    # Guardamos una copia del DataFrame original
    df_original = df.copy()
    df_dict_original[key] = df_original

    # Creamos una copia para discretización
    df_disc = df.copy()

    # Para 'speed', 'RPM' y 'Steering wheel angle' calculamos la diferencia entre muestras consecutivas.
    df_disc["Speed_trend"] = (
        df_disc["speed"].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    )
    df_disc["RPM_trend"] = (
        df_disc["RPM"].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    )
    df_disc["Steering_trend"] = (
        df_disc["Steering wheel angle"]
        .diff()
        .apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    )

    # Para 'Gas pedal' y 'Brake pedal', aplicamos el mismo criterio.
    df_disc["Gas_trend"] = (
        df_disc["Gas pedal"]
        .diff()
        .apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    )
    df_disc["Brake_trend"] = (
        df_disc["Brake pedal"]
        .diff()
        .apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    )

    # 'Clutch pedal' se conserva tal cual, ya que es binaria.
    df_disc["Clutch_trend"] = df_disc["Clutch pedal"]

    # Para 'Gear', definimos una función personalizada que tenga en cuenta que durante
    # un cambio de marcha pueden aparecer instantes con valor 0 (punto muerto) intermedios.
    def compute_gear_trend(gear_series):
        trend = []
        prev = None  # Almacena el último valor distinto de 0
        for val in gear_series:
            if prev is None:
                trend.append(0)
                if val != 0:
                    prev = val
            else:
                if val == 0:
                    # El 0 intermedio se considera parte de la transición sin modificar la tendencia.
                    trend.append(0)
                else:
                    # Comparamos con el último valor no cero.
                    if val > prev:
                        trend.append(1)
                    elif val < prev:
                        trend.append(-1)
                    else:
                        trend.append(0)
                    prev = val
        return trend

    df_disc["Gear_trend"] = compute_gear_trend(df_disc["Gear"])

    # Eliminamos las columnas originales de las señales para dejar solo las discretizadas.
    columns_to_drop = [
        "speed",
        "RPM",
        "Steering wheel angle",
        "Gas pedal",
        "Brake pedal",
        "Clutch pedal",
        "Gear",
    ]
    df_disc = df_disc.drop(columns=columns_to_drop)

    # Guardamos el DataFrame discretizado en el diccionario correspondiente.
    df_dict_discretized[key] = df_disc

# Ejemplo: Mostramos las primeras 10 filas de un DataFrame original y su versión discretizada.
example_key = ("Driver1", "STISIMData_3step-Turnings")
if example_key in df_dict_original and example_key in df_dict_discretized:
    print("Original DataFrame:")
    display(df_dict_original[example_key].head(10))

    print("Discretized DataFrame (solo columnas de tendencia e identificación):")
    display(df_dict_discretized[example_key].head(10))
else:
    print(f"No se encontró el DataFrame para {example_key}")

Original DataFrame:


,speed,RPM,Steering wheel angle,Gas pedal,Brake pedal,Clutch pedal,Gear,Maneuver marker flag
0,0.06,147.244,-8.24,0,0,0,0,0
1,0.04,184.118,-8.24,0,0,0,0,0
2,0.02,219.193,-8.24,0,0,0,0,0
3,0.00,252.554,-8.24,0,0,0,0,0
4,0.00,284.287,-8.24,0,0,0,0,0
5,0.00,314.470,-8.24,0,0,0,0,0
6,0.00,343.179,-8.24,0,0,0,0,0
7,0.00,370.486,-7.20,0,0,0,0,0
8,0.00,396.460,-8.24,0,0,0,0,0
9,0.00,425.960,-8.24,0,0,0,0,0


Discretized DataFrame (solo columnas de tendencia e identificación):


,Maneuver marker flag,Speed_trend,RPM_trend,Steering_trend,Gas_trend,Brake_trend,Clutch_trend,Gear_trend
0,0,0,0,0,0,0,0,0
1,0,-1,1,0,0,0,0,0
2,0,-1,1,0,0,0,0,0
3,0,-1,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0
5,0,0,1,0,0,0,0,0
6,0,0,1,0,0,0,0,0
7,0,0,1,1,0,0,0,0
8,0,0,1,-1,0,0,0,0
9,0,0,1,0,0,0,0,0


# 2. Segmentación Temporal de los Datos

## 2.1. Definición de Ventanas

Para analizar la dinámica de las maniobras de conducción, es necesario segmentar los datos en ventanas deslizantes. La elección del tamaño de la ventana depende de la frecuencia de muestreo y de la duración de la maniobra que queremos capturar. 

Por ejemplo, si los datos se registran a $20 \, \text{Hz}$, una ventana de 2 segundos corresponderá a 40 muestras. Para el caso del proyecto, se ha utilizado un valor de `window_size=10` (puede ser ajustado según las necesidades), lo que equivale a 10 muestras, un total de 0,5 segundos. Este valor se puede modificar para capturar ventanas más largas o cortas, dependiendo de la resolución temporal que deseemos analizar.

Existen dos enfoques para segmentar los datos:
- **Ventanas solapadas (overlapping):**  
  En este método, cada ventana se desplaza en el tiempo en pasos pequeños (por ejemplo, de 1 muestra), de modo que cada ventana comparte parte de sus datos con la ventana anterior. Esto permite capturar transiciones suaves y mejora la detección de patrones continuos, pero aumenta la cantidad de datos a procesar.

- **Ventanas no solapadas (non-overlapping):**  
  Las ventanas se extraen en bloques independientes, sin compartir datos entre ellas. Esto reduce el número total de ventanas y puede simplificar el procesamiento, aunque se pierde la continuidad entre ventanas.

La elección entre ventanas solapadas o no solapadas depende de la naturaleza del análisis y del modelo a entrenar. Para la detección de maniobras, donde la continuidad temporal es importante, las ventanas solapadas pueden ofrecer una mejor representación de la dinámica, aunque se pueden comparar ambos métodos para determinar cuál se adapta mejor a este caso.

A continuación, se muestra el código que implementa ambas estrategias:

In [6]:
def create_overlapping_time_windows(df, window_size=10):
    """
    Crea ventanas temporales solapadas.

    Parámetros:
    - df: DataFrame con los datos.
    - window_size: Número de muestras por ventana.

    Devuelve:
    - features: Array de características, donde cada fila es una ventana aplanada (sin la columna 'Maneuver marker flag').
    - labels: Array de etiquetas, obtenida del valor de 'Maneuver marker flag' inmediatamente después de cada ventana.
    """
    features, labels = [], []
    for i in range(len(df) - window_size):
        window = df.iloc[i : i + window_size].drop(columns=["Maneuver marker flag"])
        features.append(window.values.flatten())
        labels.append(df.iloc[i + window_size]["Maneuver marker flag"])
    return np.array(features), np.array(labels)


def create_non_overlapping_time_windows(df, window_size=10):
    """
    Crea ventanas temporales no solapadas.

    Parámetros:
    - df: DataFrame con los datos.
    - window_size: Número de muestras por ventana.

    Devuelve:
    - features: Array de características, donde cada fila es una ventana aplanada (sin la columna 'Maneuver marker flag').
    - labels: Array de etiquetas, obtenida del valor de 'Maneuver marker flag' inmediatamente después de cada ventana.
    """
    features, labels = [], []
    for i in range(0, len(df) - window_size, window_size):
        window = df.iloc[i : i + window_size].drop(columns=["Maneuver marker flag"])
        features.append(window.values.flatten())
        labels.append(df.iloc[i + window_size]["Maneuver marker flag"])
    return np.array(features), np.array(labels)

## 2.2. Creación de los conjuntos de entrenamiento y test

A partir de este punto, se procederá a guardar los conjuntos de datos de entrenamiento y prueba generados a partir de distintas configuraciones de preprocesamiento. Estas configuraciones dependen de los siguientes factores:

1. **Overlapping en ventanas deslizantes:** Se considerarán versiones con y sin solapamiento de las ventanas.
2. **Tamaño de la ventana:** Diferentes tamaños de ventana afectarán la cantidad de información contenida en cada muestra, $n ∈ \{5, 10, 15, 20\}$.
3. **Método de escalado aplicado:** Se explorarán diferentes técnicas de normalización y estandarización:
   - **Estandarización (`StandardScaler()`)**: Transforma los datos para que tengan media 0 y desviación estándar 1.
   - **Min-Max Scaling (`MinMaxScaler()`)**: Escala los valores en un rango $n ∈ \{0..1\}$.
   - **Discretización**: Convierte los datos en categorías discretas basadas en intervalos predefinidos.

Cada conjunto de datos generado se almacenará de forma estructurada y etiquetada para facilitar su identificación y posterior uso en la predicción de maniobras en vehículos.


In [ ]:
window_sizes = [5, 10, 15, 20]
scaling_methods = ["discrete", "standard", "minmax"]
segmentation_types = ["overlapping", "non_overlapping"]

# Diccionario para almacenar los resultados:
# results[segmentation_type][window_size][scaling_method][maneuver] = {X_train, y_train, X_test, y_test}
results = {}

# Iteramos por cada tipo de segmentación
for seg_type in segmentation_types:
    results[seg_type] = {}
    # Seleccionamos la función de segmentación
    if seg_type == "overlapping":
        seg_func = create_overlapping_time_windows
    else:
        seg_func = create_non_overlapping_time_windows

    # Para cada tamaño de ventana
    for window_size in window_sizes:
        results[seg_type][window_size] = {}
        # Para cada método de escalado
        for scale_method in scaling_methods:
            results[seg_type][window_size][scale_method] = {}
            # Seleccionamos la fuente de datos según el método de escalado:
            # "discrete" utiliza los DataFrames ya discretizados; los otros usan los datos originales.
            if scale_method == "discrete":
                source_dict = df_dict_discretized
            else:
                source_dict = df_dict_original

            # Agrupamos los DataFrames por maniobra (cada clave es (Driver, Maneuver))
            maneuvers = {}
            for key, df in source_dict.items():
                driver, maneuver = key
                if maneuver not in maneuvers:
                    maneuvers[maneuver] = {}
                maneuvers[maneuver][driver] = df

            # Para cada maniobra, generamos las ventanas y separamos en train (Drivers 1-4) y test (Driver5)
            for maneuver, driver_dict in maneuvers.items():
                X_train_list, y_train_list = [], []
                X_test_list, y_test_list = [], []

                # Datos de entrenamiento: Driver1 a Driver4
                for driver in ["Driver1", "Driver2", "Driver3", "Driver4"]:
                    if driver in driver_dict:
                        df_driver = driver_dict[driver]
                        features, labels = seg_func(df_driver, window_size=window_size)
                        X_train_list.append(features)
                        y_train_list.append(labels)

                # Datos de test: Driver5
                if "Driver5" in driver_dict:
                    df_driver = driver_dict["Driver5"]
                    features, labels = seg_func(df_driver, window_size=window_size)
                    X_test_list.append(features)
                    y_test_list.append(labels)

                X_train = np.concatenate(X_train_list, axis=0) if X_train_list else None
                y_train = np.concatenate(y_train_list, axis=0) if y_train_list else None
                X_test = np.concatenate(X_test_list, axis=0) if X_test_list else None
                y_test = np.concatenate(y_test_list, axis=0) if y_test_list else None

                # Aplicamos escalado para "standard" y "minmax" (para "discrete" dejamos los datos tal y como están)
                if scale_method == "standard" and X_train is not None:
                    scaler = StandardScaler()
                    X_train = scaler.fit_transform(X_train)
                    X_test = scaler.transform(X_test)
                elif scale_method == "minmax" and X_train is not None:
                    scaler = MinMaxScaler()
                    X_train = scaler.fit_transform(X_train)
                    X_test = scaler.transform(X_test)

                # Guardamos los datos para esta maniobra
                results[seg_type][window_size][scale_method][maneuver] = {
                    "X_train": X_train,
                    "y_train": y_train,
                    "X_test": X_test,
                    "y_test": y_test,
                }

print("Segmentación y preparación de conjuntos completadas.")

Segmentación y preparación de conjuntos completadas.


In [8]:
# Verificación: Imprimir las dimensiones de los conjuntos para cada combinación de segmentación, ventana, escalado y maniobra

for seg_type in results:
    print(f"--- Segmentación: {seg_type} ---")
    for window_size in results[seg_type]:
        print(f"  Ventana de tamaño: {window_size}")
        for scale_method in results[seg_type][window_size]:
            print(f"    Método de escalado: {scale_method}")
            for maneuver, data in results[seg_type][window_size][scale_method].items():
                X_train, y_train = data["X_train"], data["y_train"]
                X_test, y_test = data["X_test"], data["y_test"]
                if X_train is not None and X_test is not None:
                    print(f"      Maniobra: {maneuver}")
                    print(
                        f"        X_train shape: {X_train.shape}, y_train shape: {y_train.shape}"
                    )
                    print(
                        f"        X_test shape: {X_test.shape}, y_test shape: {y_test.shape}"
                    )
            print()
    print()

--- Segmentación: overlapping ---
  Ventana de tamaño: 5
    Método de escalado: discrete
      Maniobra: STISIMData_3step-Turnings
        X_train shape: (39045, 35), y_train shape: (39045,)
        X_test shape: (8657, 35), y_test shape: (8657,)
      Maniobra: STISIMData_Overtaking
        X_train shape: (25354, 35), y_train shape: (25354,)
        X_test shape: (6178, 35), y_test shape: (6178,)
      Maniobra: STISIMData_Stopping
        X_train shape: (47715, 35), y_train shape: (47715,)
        X_test shape: (11546, 35), y_test shape: (11546,)
      Maniobra: STISIMData_Turnings
        X_train shape: (34400, 35), y_train shape: (34400,)
        X_test shape: (11276, 35), y_test shape: (11276,)
      Maniobra: STISIMData_U-Turnings
        X_train shape: (24511, 35), y_train shape: (24511,)
        X_test shape: (5979, 35), y_test shape: (5979,)

    Método de escalado: standard
      Maniobra: STISIMData_3step-Turnings
        X_train shape: (39045, 35), y_train shape: (39045,)


# 3. Definición del Problema de Clasificación/Detección

El objetivo de esta fase es desarrollar un modelo capaz de clasificar o detectar, en cada ventana temporal, si se está ejecutando una maniobra determinada. Para ello, definimos el problema de la siguiente forma:

- **Objetivo:**  
  Clasificar, para cada ventana de datos, si la maniobra está activa (clasificación binaria: 1 para maniobra activa, 0 para inactiva).

- **Variables de Entrada:**  
  Las variables de entrada son las características extraídas de cada ventana temporal. Estas características pueden provenir de:
  - Los datos **discretizados** (por ejemplo, las tendencias: Speed_trend, RPM_trend, Steering_trend, Gas_trend, Brake_trend, Clutch_trend, Gear_trend), que ya reflejan la dinámica de la conducción.
  - O bien, de los datos escalados (Standard o MinMax) aplicados a las señales originales, según se requiera en cada experimento.

  Cada ventana se aplanará en un vector, de forma que cada ejemplo representa la evolución de las señales durante ese intervalo.

- **Variable Objetivo:**  
  La etiqueta de cada ventana se obtiene a partir de la columna *Maneuver marker flag*. En el caso binario:
  - **Clase Positiva (1):** La maniobra está activa en la ventana.
  - **Clase Negativa (0):** La maniobra no está activa.

- **División de los Datos:**  
  Los conjuntos se han preparado de forma jerárquica:
  - Se han generado dos grandes conjuntos: uno a partir de ventanas **overlapping** y otro a partir de ventanas **non-overlapping**.
  - Para cada conjunto, se han aplicado tres escalados distintos:  
    - **Discretización:** Usando los datos ya discretizados.
    - **Standard Scaling:** Normalizando para que las características tengan media 0 y desviación 1.
    - **MinMax Scaling:** Escalando las características al rango [0, 1].
  - Además, se han generado ventanas con distintos tamaños $n ∈ \{5, 10, 15, 20\}$.
  - Finalmente, para cada combinación se han preparado los conjuntos de **entrenamiento** y **test** de la siguiente manera:  
    - **Entrenamiento:** Ventanas de los conductores Driver1, Driver2, Driver3 y Driver4 (concatenadas en orden para cada maniobra).
    - **Test:** Ventanas del conductor Driver5, que se reservan para evaluar la capacidad del modelo de generalizar a datos de un conductor distinto.

Este planteamiento nos permite evaluar cómo influyen el tipo de segmentación, el tamaño de la ventana y la técnica de escalado en el rendimiento del modelo.

In [9]:
# Ejemplo: Mostramos las dimensiones de los conjuntos para una combinación específica:
# Segmentación: overlapping, Tamaño de ventana: 10, Escalado: standard, para cada maniobra.

example_seg_type = "overlapping"
example_window_size = 10
example_scale = "standard"

print(
    f"--- Resultados para segmentación: {example_seg_type}, ventana: {example_window_size}, escalado: {example_scale} ---\n"
)
for maneuver, data in results[example_seg_type][example_window_size][
    example_scale
].items():
    X_train, y_train = data["X_train"], data["y_train"]
    X_test, y_test = data["X_test"], data["y_test"]
    if X_train is not None and X_test is not None:
        print(f"Maniobra: {maneuver}")
        print(f"  X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"  X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")

--- Resultados para segmentación: overlapping, ventana: 10, escalado: standard ---

Maniobra: STISIMData_3step-Turnings
  X_train shape: (39025, 70), y_train shape: (39025,)
  X_test shape: (8652, 70), y_test shape: (8652,)

Maniobra: STISIMData_Overtaking
  X_train shape: (25334, 70), y_train shape: (25334,)
  X_test shape: (6173, 70), y_test shape: (6173,)

Maniobra: STISIMData_Stopping
  X_train shape: (47695, 70), y_train shape: (47695,)
  X_test shape: (11541, 70), y_test shape: (11541,)

Maniobra: STISIMData_Turnings
  X_train shape: (34380, 70), y_train shape: (34380,)
  X_test shape: (11271, 70), y_test shape: (11271,)

Maniobra: STISIMData_U-Turnings
  X_train shape: (24491, 70), y_train shape: (24491,)
  X_test shape: (5974, 70), y_test shape: (5974,)



# 4. Construcción y Entrenamiento del Modelo

## 4.1. Selección del Modelo

En esta sección se justifica la elección del algoritmo de IA para la detección de maniobras. Para este problema, se ha optado por utilizar un clasificador basado en Random Forest (RF), combinado con una búsqueda en rejilla (Grid Search) para la optimización de hiperparámetros.

**Razones para elegir Random Forest:**
- **Robustez y Precisión:**  
  Random Forest es un método de ensamblado que combina múltiples árboles de decisión, lo que ayuda a reducir el sobreajuste y mejora la precisión en problemas de clasificación.
  
- **Capacidad para Manejar Datos de Alta Dimensionalidad y No Lineales:**  
  Debido a que los datos provienen de ventanas deslizantes con múltiples características (por ejemplo, tendencias discretizadas de las señales), Random Forest es capaz de capturar relaciones complejas y no lineales entre las variables.
  
- **Facilidad de Interpretación y Ajuste:**  
  A su vez, mediante el uso de Grid Search, se ha explorado sistemáticamente diferentes combinaciones de hiperparámetros (como el número de árboles, la profundidad máxima de los árboles y el número mínimo de muestras) para optimizar el rendimiento del modelo, sin necesidad de realizar ajustes manuales extensivos.
  
En resumen, Random Forest combinado con Grid Search ofrece un enfoque robusto y eficiente para clasificar si una maniobra está activa o no en cada ventana deslizante, lo que se alinea con los objetivos del proyecto.

# 4.2. Implementación del Modelo

En este apartado, se construye y entrena los modelos de detección para cada maniobra utilizando Random Forest y Grid Search para la optimización de hiperparámetros. Se realiza el Grid Search de forma independiente para cada conjunto de datos, diferenciando entre:

- **Tipo de segmentación:** con y sin solapamiento.
- **Tamaño de la ventana:** 5, 10, 15 y 20 muestras.
- **Método de escalado:** discrete, standard y minmax.
- **Por cada maniobra:** Los datos de entrenamiento se forman concatenando las ventanas de Driver1 a Driver4, y los datos de test provienen de Driver5.

Esta separación permite evaluar cómo influyen la segmentación, el tamaño de la ventana y el escalado en el rendimiento del modelo.

In [12]:
# Definición del grid de hiperparámetros para Random Forest
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
}

# Creamos las carpetas para guardar los resultados de forma estructurada
csv_dir = os.path.join("results", "csv")
models_dir_overlapping = os.path.join("results", "models", "overlapping")
models_dir_non_overlapping = os.path.join("results", "models", "non_overlapping")
os.makedirs(csv_dir, exist_ok=True)
os.makedirs(models_dir_overlapping, exist_ok=True)
os.makedirs(models_dir_non_overlapping, exist_ok=True)

# Inicializamos listas y diccionarios para almacenar resultados y mejores modelos
evaluation_results = (
    []
)  # Para almacenar cada resultado y posteriormente crear un DataFrame global
best_f1_dict = {"overlapping": {}, "non_overlapping": {}}
best_models_overlapping = {}
best_models_non_overlapping = {}

# Iteramos por cada combinación en el diccionario 'results'
for seg_type in results:
    for window_size in results[seg_type]:
        for scale_method in results[seg_type][window_size]:
            for maneuver, data in results[seg_type][window_size][scale_method].items():
                X_train = data["X_train"]
                y_train = data["y_train"]
                X_test = data["X_test"]
                y_test = data["y_test"]

                # Instanciamos el clasificador Random Forest y configuramos Grid Search con CV=3
                rf = RandomForestClassifier(random_state=42)
                grid_search = GridSearchCV(
                    estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring="f1"
                )
                grid_search.fit(X_train, y_train)
                best_rf = grid_search.best_estimator_
                best_params = grid_search.best_params_

                # Predicción y cálculo de métricas en el conjunto de test
                y_pred = best_rf.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)

                # Creamos la clave para identificar esta combinación
                key = (seg_type, window_size, scale_method, maneuver)
                print(
                    f"Segmentación: {seg_type}, Ventana: {window_size}, Escalado: {scale_method}, Maniobra: {maneuver}"
                )
                print(
                    f"  -> F1: {f1:.4f} | Acc: {accuracy:.4f} | Prec: {precision:.4f} | Recall: {recall:.4f}\n"
                )

                # Guardamos los resultados de esta combinación en un diccionario
                result_dict = {
                    "Segmentation": seg_type,
                    "Window Size": window_size,
                    "Scaler": scale_method,
                    "Maneuver": maneuver,
                    "Best Params": best_params,
                    "Accuracy": accuracy,
                    "Precision": precision,
                    "Recall": recall,
                    "F1 Score": f1,
                }
                evaluation_results.append(result_dict)

                # Guardamos los resultados en un CSV específico para cada maniobra
                csv_filename = os.path.join(
                    csv_dir, f"evaluation_results_{maneuver}.csv"
                )
                df_row = pd.DataFrame([result_dict])
                if not os.path.exists(csv_filename):
                    df_row.to_csv(csv_filename, index=False)
                else:
                    df_row.to_csv(csv_filename, mode="a", index=False, header=False)

                # Actualizamos y guardamos el mejor modelo para cada maniobra, según el tipo de segmentación.
                if seg_type == "overlapping":
                    if (maneuver not in best_f1_dict["overlapping"]) or (
                        f1 > best_f1_dict["overlapping"][maneuver]
                    ):
                        best_f1_dict["overlapping"][maneuver] = f1
                        best_models_overlapping[maneuver] = best_rf
                        model_filename = os.path.join(
                            models_dir_overlapping,
                            f"best_model_overlapping_{maneuver}.pkl",
                        )
                        with open(model_filename, "wb") as f:
                            pickle.dump(best_rf, f)
                        print(
                            f"Guardado mejor modelo OVERLAPPING para {maneuver} (F1: {f1:.4f})\n"
                        )
                elif seg_type == "non_overlapping":
                    if (maneuver not in best_f1_dict["non_overlapping"]) or (
                        f1 > best_f1_dict["non_overlapping"][maneuver]
                    ):
                        best_f1_dict["non_overlapping"][maneuver] = f1
                        best_models_non_overlapping[maneuver] = best_rf
                        model_filename = os.path.join(
                            models_dir_non_overlapping,
                            f"best_model_non_overlapping_{maneuver}.pkl",
                        )
                        with open(model_filename, "wb") as f:
                            pickle.dump(best_rf, f)
                        print(
                            f"Guardado mejor modelo NON-OVERLAPPING para {maneuver} (F1: {f1:.4f})\n"
                        )

# Convertimos evaluation_results en un DataFrame global y lo guardamos en un CSV global
global_results_df = pd.DataFrame(evaluation_results)
global_csv_path = os.path.join("results", "global_evaluation_results.csv")
global_results_df.to_csv(global_csv_path, index=False)
print(
    "Todos los resultados han sido guardados y los mejores modelos han sido actualizados por maniobra."
)

Segmentación: overlapping, Ventana: 5, Escalado: discrete, Maniobra: STISIMData_3step-Turnings
  -> F1: 0.6809 | Acc: 0.6710 | Prec: 0.8852 | Recall: 0.5532

Guardado mejor modelo OVERLAPPING para STISIMData_3step-Turnings (F1: 0.6809)

Segmentación: overlapping, Ventana: 5, Escalado: discrete, Maniobra: STISIMData_Overtaking
  -> F1: 0.5965 | Acc: 0.6588 | Prec: 0.8200 | Recall: 0.4687

Guardado mejor modelo OVERLAPPING para STISIMData_Overtaking (F1: 0.5965)

Segmentación: overlapping, Ventana: 5, Escalado: discrete, Maniobra: STISIMData_Stopping
  -> F1: 0.5914 | Acc: 0.7426 | Prec: 0.4937 | Recall: 0.7374

Guardado mejor modelo OVERLAPPING para STISIMData_Stopping (F1: 0.5914)

Segmentación: overlapping, Ventana: 5, Escalado: discrete, Maniobra: STISIMData_Turnings
  -> F1: 0.7953 | Acc: 0.8272 | Prec: 0.8851 | Recall: 0.7220

Guardado mejor modelo OVERLAPPING para STISIMData_Turnings (F1: 0.7953)

Segmentación: overlapping, Ventana: 5, Escalado: discrete, Maniobra: STISIMData_U-Tu